# <center> Generators

Let's start from the basis: what is an **iterator**? <ins>Iterators are objects that can be iterated over</ins>. We can also say that an <ins>iterator is an object, which returns data, one element at a time</ins>. Also, they do not do any work until we explicitly ask for their next item.  
They work on a principle, which is known in computer science as **lazy evaluation**. <ins>Lazy evaluation is an evaluation strategy which delays the evaluation of an expression until its value is really needed</ins>. Due to the laziness of Python iterators, they are a great way to deal with infinity, i.e. iterables which can iterate for ever.

You already learned in your first Python programs that you can iterate over container objects such as lists and strings. To do this, Python creates an iterator version of the list or string. In this case, an iterator can be seen as a pointer to a container, which enables us to iterate over all the elements of this container. An iterator is an abstraction, which enables the programmer to access all the elements of an iterable object (a set, a string, a list etc.) without any deeper knowledge of the data structure of this object.

**Generators** are a special kind of function, which enable us to implement or generate iterators.
    
Note: the material used in this notebook has been taken from: [link](https://www.python-course.eu/python3_generators.php).

## 1. Iterators
### 1.1 Transforming a list into an iterable

In [ ]:
expertises = ["Python Beginner", 
              "Python Intermediate", 
              "Python Proficient", 
              "Python Advanced"]
expertises_iterator = iter(expertises)  # If it is not iterable, an exception will be raised
print("Calling 'next' for the first time: ", next(expertises_iterator))
print("Calling 'next' for the second time: ", next(expertises_iterator))

**NOTE**: an iterable has a **state** that remembers the last item visited. Once you completed the loop, you cannot call ```next(iterable)``` again, or you will get an error.

In [ ]:
other_cities = ["Strasbourg", "Freiburg", "Stuttgart", 
                "Vienna / Wien", "Hannover", "Berlin", 
                "Zurich"]

city_iterator = iter(other_cities)
while city_iterator:
    try:
        city = next(city_iterator)
        print(city)
    except StopIteration:  # This prevent from getting a crash when you reach the end of the iterable
        break

### 1.2 Implementing an iterable
One way to create iterators in Python is defining a class which implements the methods **\_\_init\_\_** and **\_\_next\_\_**.  
We show this by implementing a class cycle, which can be used to cycle over an iterable object forever. In other words, an instance of this class returns the element of an iterable until it is exhausted. Then it repeats the sequence indefinitely.  
NOTE: this is nice, but the usual and easiest way to create an iterator in Python consists in using a generator function.

In [ ]:
class Cycle(object):
    
    def __init__(self, iterable):  # You define the class by passing an iterable to the constructor
        self.iterable = iterable
        self.iter_obj = iter(iterable)

    def __iter__(self):  # Needed is your class is an iterable
        return self

    def __next__(self):  # Allow you to be able to call next(x) for an instanxe x of the class Cycle
        while True:
            try:
                next_obj = next(self.iter_obj)
                return next_obj  # next(x) will retunr the next item
            except StopIteration:
                self.iter_obj = iter(self.iterable)  # Once you reach the end you initialize the iterable to the beginning

      
x = Cycle("abc")

for i in range(10):
    print(next(x), end=", ")

## 2. Generators
**Generators** look like functions, but there is both a syntactic and a semantic difference.   
One distinguishing characteristic is the yield statements. The **yield statement** turns a functions into a generator. <ins>A generator is a function which returns a generator object</ins>. This generator object can be seen like a function which produces a sequence of results instead of a single object. This sequence of values is produced by iterating over it, e.g. with a for loop. The values, on which can be iterated, are created by using the yield statement. The value created by the yield statement is the value following the yield keyword. <ins>The execution of the code stops when a yield statement is reached</ins>. The value behind the yield will be returned.  
As soon as "next" is called again on the generator object, the generator function will resume execution right after the yield statement in the code, where the last call is made. <ins>The execution will continue in the state in which the generator was left</ins> after the last yield. In other words, all the local variables still exist, because they are automatically saved between calls. This is a fundamental difference to functions: functions always start their execution at the beginning of the function body, regardless of where they had left in previous calls. They don't have any static or persistent values. There may be more than one yield statement in the code of a generator or the yield statement might be inside the body of a loop.   
If there is a return statement in the code of a generator, the execution will stop with a StopIteration exception error when this code is executed by the Python interpreter. The word "generator" is sometimes ambiguously used to mean both the generator function itself and the objects which are generated by a generator.

Everything which can be done with a generator can also be implemented with a class based iterator as well. However, the crucial advantage of generators consists in automatically creating the methods __iter__() and next(). Generators provide a very neat way of producing data which is huge or even infinite.

In [ ]:
# The following is a simple example of a generator, which is capable of producing various city names.
def city_generator():
    yield("Hamburg")
    yield("Konstanz")
    yield("Berlin")
    yield("Zurich")
    yield("Schaffhausen")
    yield("Stuttgart")  
city = city_generator()
print(next(city))
print(next(city))

Once again if we reach ther end of the generator we get an error. The only way to start looping from the beginning it is to call ```city = city_generator()``` again.

### 2.1 Method of Operation
Generators offer a comfortable method to generate iterators, and that's why they are called generators.
Method of working:  
*  A generator is called like a function. Its return value is an iterator, i.e. a generator object. The code of the generator will not be executed at this stage.  
* The iterator can be used by calling the next method. The first time the execution starts like a function, i.e. the first line of code within the body of the iterator. The code is executed until a yield statement is reached.  
* yield returns the value of the expression, which is following the keyword yield. This is like a function, but Python keeps track of the position of this yield and the state of the local variables is stored for the next call. At the next call, the execution continues with the statement following the yield statement and the variables have the same values as they had in the previous call.  
* The iterator is finished, if the generator body is completely worked through or if the program flow encounters a return statement without a value.  

We will illustrate this behaviour in the following example. The generator count creates an iterator which creates a sequence of values by counting from the start value 'firstval' and using 'step' as the increment for counting:



In [ ]:
# The generator count creates an iterator which creates a sequence of values by counting
# from the start value 'firstval' and using 'step' as the increment for counting

def count(firstval=0, step=1):
    x = firstval  # The fiurst time you call next you start here.
    while True:
        yield x  # Then you stop here and return x.
        x += step  # When you use next again you will resume from here and stop at the yield again.
        
counter = count() # count will start with 0
for i in range(10):
    print(next(counter), end=", ")

start_value = 2.1
stop_value = 0.3
print("\nNew counter:")
counter = count(start_value, stop_value)
for i in range(10):
    new_value = next(counter)
    print(f"{new_value:2.2f}", end=", ")

NOTE: a generator need to have at least a ```yield``` statement.   It can have a ```return``` but it will behave as a ```raise StopIteration()```.

### 2.2 Send() method
**Generators can** not only send objects but **also receive objects** . 
 <ins>Sending</ins> a message, i.e. an object, into the generator <ins>can be achieved by applying the send method to the generator object</ins>.   Be aware of the fact that <ins>send both sends a value to the generator and returns the value yielded by the generator</ins> .   
We will demonstrate this behavior in the following simple example of a coroutine:

In [ ]:
def count(firstval=0, step=1):
    counter = firstval
    while True:
        # This line allow you to receive what you send with send() and store it into 'new_counter_val'
        new_counter_val = yield counter  # If you only use 'next(counter)' you are sending a None.
        if new_counter_val is None:
            counter += step
        else:  # If you have counter.send(100.5) you start here and you stop at the next yield.
            counter = new_counter_val
            
start_value = 2.1
stop_value = 0.3
counter = count(start_value, stop_value) 
for i in range(10):
    new_value = next(counter)
    print(f"{new_value:2.2f}", end=", ")
 
print("set current count value to another value:")
tmp = counter.send(100.5)
print(f'{tmp} is what you received from the counter.send(100.5)')
for i in range(10):
    new_value = next(counter)
    print(f"{new_value:2.2f}", end=", ")
    

### 2.3 Throw() method
The ```throw()``` method <ins>raises an exception at the point where the generator was paused, and returns the next value yielded by the generator</ins>. It raises **StopIteration** <ins>if the generator exits without yielding another value</ins>. The generator has to catch the passed-in exception, otherwise the exception will be propagated to the caller.

The infinite_looper from our previous example keeps yielding the elements of the sequential data, but we don't have any information about the index or the state of the variable "count". We can get this information by throwing an exception with the "throw" method. We catch this exception inside of the generator and print the value of "count".

In [ ]:
def count(firstval=0, step=1):
    counter = firstval
    while True:
        try:
            new_counter_val = yield counter
            if new_counter_val is None:
                counter += step
            else:
                counter = new_counter_val
        except Exception:
            yield (firstval, step, counter)

c = count()
for i in range(3):
    print(next(c))  # So far it worked as a normal generator
print("Let us see what the state of the iterator is:")
i = c.throw(Exception)  # It raises an Exception, so it goes at the line after 'except Exception:'
print(i)
print("now, we can continue:")
for i in range(3):
    print(next(c))  # The next number you print is still 2 because the next yield after the exception has no increment.

We can improve the previous example by defining our own exception class StateOfGenerator

In [ ]:
class StateOfGenerator(Exception):
     def __init__(self, message=None):
         self.message = message

def count(firstval=0, step=1):
    counter = firstval
    while True:
        try:
            new_counter_val = yield counter
            if new_counter_val is None:
                counter += step
            else:
                counter = new_counter_val
        except StateOfGenerator:  # This is simply a cleaner version of the previous example.
            yield (firstval, step, counter)

            
c = count()
for i in range(3):
    print(next(c))
print("Let us see what the state of the iterator is:")
i = c.throw(StateOfGenerator)
print(i)
print("now, we can continue:")
for i in range(3):
    print(next(c))

### 2.4 Yield From
The **yield from < expr >** statement can be used inside the body of a generator. <ins> Expr has to be an expression evaluating to an iterable, from which an iterator will be extracted</ins>. The iterator is run to exhaustion, i.e. until it encounters a StopIteration exception. This iterator yields and receives values to or from the caller of the generator, i.e. the one which contains the yield from statement.

We can learn from the following example by looking at the two generators 'gen1' and 'gen2' that yield from is substituting the for loops of 'gen1':

In [ ]:
def gen1():
    for char in "Python":
        yield char
    for i in range(5):
        yield i

def gen2():
    yield from "Python"  # The same that gen1 does
    yield from range(5)  # The same that gen1 does

g1, g2 = gen1(), gen2()

print("g1: ", end=", ")
for x in g1:
    print(x, end=", ")
print("\ng2: ", end=", ")
for x in g2:
    print(x, end=", ")
print()

The benefit of a yield from statement can be seen as a way to split a generator into multiple generators.  
That's what we have done in our previous example and we will demonstrate this more explicitely in the following example:

In [ ]:
def cities():  # Subgenerator1
    for city in ["Berlin", "Hamburg", "Munich", "Freiburg"]:
        yield city

def squares(): # Subgenerator2
    for number in range(10):
        yield number ** 2
        
def generator_v1():
    for city in cities():
        yield city
    for number in squares():
        yield number
        
def generator_v2():
    yield from cities()
    yield from squares()
    
lst1 = [el for el in generator_v1()]
lst2 = [el for el in generator_v2()]
print(lst1 == lst2)  # They are the same!

If the < expr > from the yield from is another generator, the effect is the same as if the body of the sub‐generator were inlined at the point of the yield from statement.  
NOTE: the <ins>subgenerator is allowed to execute a return statement with a value</ins>, and that value becomes the value of the yield from expression:

In [ ]:
def subgenerator():
    yield 1
    return 42

def delegating_generator():
    x = yield from subgenerator()
    print(x)

for x in delegating_generator():
    print(x)

The full semantics of the yield from expression is described in six points in "PEP 380 -- Syntax for Delegating to a Subgenerator" in terms of the generator protocol:

* Any values that the iterator yields are passed directly to the caller.  
* Any values sent to the delegating generator using send() are passed directly to the iterator. If the sent value is None, the iterator's next() method is called. If the sent value is not None, the iterator's send() method is called. If the call raises StopIteration, the delegating generator is resumed. Any other exception is propagated to the delegating generator.  
* Exceptions other than GeneratorExit thrown into the delegating generator are passed to the throw() method of the iterator. If the call raises StopIteration, the delegating generator is resumed. Any other exception is propagated to the delegating generator.  
* If a GeneratorExit exception is thrown into the delegating generator, or the close() method of the delegating generator is called, then the close() method of the iterator is called if it has one. If this call results in an exception, it is propagated to the delegating generator. Otherwise, GeneratorExit is raised in the delegating generator.  
* The value of the yield from expression is the first argument to the StopIteration exception raised by the iterator when it terminates.  
* return expr in a generator causes StopIteration(expr) to be raised upon exit from the generator.  


### 2.5 Recursive Generators
A permutation is a rearrangement of the elements of an ordered list. In other words: every arrangement of n elements is called a permutation. In the following lines we show you all the permutations of the letter a, b and c:
a b c  
a c b  
b a c  
b c a  
c a b  
c b a  
The number of permutations on a set of n elements is given by n!

Permutations, like functions or generators can be recursively programmed. The following example is a generator to create all the permutations of a given list of items. The permutation generator can be called with an arbitrary list of objects. The iterator returned by this generator generates all the possible permutations.

In [ ]:
def permutations(items):  # Generator that returns an iterator with all permutations of items
    n = len(items)
    if n==0: yield []
    else:
        for i in range(len(items)):
            for cc in permutations(items[:i]+items[i+1:]):
                yield [items[i]]+cc

for p in permutations(['r','e','d']):
    print(''.join(p))
for p in permutations(list("game")):
    print(''.join(p) + ", ", end="")

In [ ]:
# simpler way with iertools
import itertools
perms = itertools.permutations(['r','e','d'])
perms
list(perms)

### 2.6 A Generator of Generators
The second generator of our Fibonacci sequence example generates an iterator, which can theoretically produce all the Fibonacci numbers, i.e. an infinite number. But you shouldn't try to produce all these numbers with the following line.  
```list(fibonacci())```  
This will show you very fast the limits of your computer. In most practical applications, we only need the first n elements of an "endless" iterator. We can use another generator, in our example first n, to create the first n elements of a generator generator:

In [ ]:
def firstn(generator, n):  # Generator that given an input generator returns the first n items
    g = generator()
    for i in range(n):
        yield next(g)

def fibonacci():  # This is an endless generator returning Fibonacci series
    """ A Fibonacci number generator """
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

print(list(firstn(fibonacci, 10)))   